In [37]:
#problem 2 part 1 & 2
with open("problem2.txt", "r") as ins:
    array = []
    for line in ins:
        array.append(line)
big_surface = 0
all_ribbons =0
for wl in array:
    wl = wl.split('x')
    wl = [int(x) for x in wl]
    surface = 2 * wl[0] * wl[1] + 2 * wl[1] * wl[2] + 2 * wl[2] * wl[0]
    wl = sorted(wl)
    small_side = wl[0] * wl[1]
    bow = wl[0] * wl[1] * wl[2]
    ribbons = wl[0] + wl[0] + wl[1] + wl[1]
    big_surface = big_surface + surface + small_side
    all_ribbons = all_ribbons + bow + ribbons
    
print('Whole surface is:', big_surface, 'elves would also need this much of ribbon:', all_ribbons)



Whole surface is: 1598415 elves would also need this much of ribbon: 3812909


In [33]:
#problem 1 part 2 # find the instruction first leading into basement
floor = 0
with open('problem1.txt', 'r') as ins:
    for i, char in enumerate(ins.read(), 1):
        if char == '(':
            floor += 1
        else:
            floor -= 1
        if floor < 0:
            break
print(i)

1797


In [ ]:
#problem 3 part 2
# The number of people (or robots) delivering
n = 2
# Add a set of coordinates for each person.
coords = [(0,0)] * n
# Keep track of the houses we've visited
houses = set([(0,0)])

with open('prob3.txt') as f:
    chars = f.read()
    # 'Partition' the list into sets of instructions of length n
    chunks = [chars[i:i+n] for i in range(0, len(chars), n)]
    # For each set of instructions
    for chunk in chunks:
        for i, instruction in enumerate(chunk):
            # Get the current coordinates for each person for each instruction
            x, y = coords[i]
            # Adjust position
            if instruction == '^':
                y -= 1
            elif instruction == '>':
                x += 1
            elif instruction == 'v':
                y += 1
            elif instruction == '<':
                x -= 1
            # Add new house
            houses.add((x,y))
            # Store position for next round
            coords[i] = (x, y)

# Count up all the houses.
print(len(houses))

In [ ]:
#http://adventofcode.com/
#PROBLEM 5
import re

p1 = re.compile('(.*[aeiou].*){3,}') #3 vowels
p2 = re.compile("([a-z])\\1") # double letters
p3 = re.compile("ab|cd|pq|xy") #none of these

with open("problem5.txt", 'r') as ins:
        print(len([l for l in ins
                  if (not p3.search(l)) and p1.search(l) and p2.search(l)]))

p4 = re.compile("([a-z][a-z]).*\\1") 
p5 = re.compile("([a-z])[a-z]\\1") 
with open('problem5.txt', 'r') as f:
    print(len([l for l in f if p4.search(l) and p5.search(l)]))


In [ ]:
#problem 6
import re
p6_re = re.compile("([0-9]+)") # get all coordinates
lights_on = set()
with open("protblem6.txt", "r") as ins:
    for line in ins:
        x1, y1, x2, y2 = [int(x) for x in p6_re.findall(line)] ## get ranges
        if line.startswith("turn on"):
            for x in range(x1, x2+1):
                for y in range(y1, y2+1):
                    lights_on.add((x,y))
        elif line.startswith("toggle"):
            for x in range(x1, x2+1):
                for y in range(y1, y2+1):
                    if (x,y) in lights_on:
                        lights_on.remove((x,y))
                    else:
                        lights_on.add((x,y))
        elif line.startswith("turn off"):
            for x in range(x1, x2+1):
                for y in range(y1, y2+1):
                    if (x,y) in lights_on:
                        lights_on.remove((x,y))
print(len(lights_on)) #part 1


import re
from collections import defaultdict
# This regex pulls out all contiguous numbers from a line.
r = re.compile("([0-9]+)")

# Keep track of lights as a dict with 0 as a default value.
lights = defaultdict(int)
with open('protblem6.txt') as f:
    for line in f:
        # Get the ranges from the instruction via regex
        x1, y1, x2, y2 = [int(x) for x in r.findall(line)]
        # For each instruction, iterate over the range and do the right thing.
        if line.startswith('toggle'):
            for x in range(x1, x2 + 1):
                for y in range(y1, y2 + 1):
                    # Toggle: Add two to existing brightness
                    lights[(x,y)] = lights[(x,y)] + 2
        elif line.startswith('turn off'):
            for x in range(x1, x2 + 1):
                for y in range(y1, y2 + 1):
                    # Turn off: Remove one from existing brightness, but not below 0
                    lights[(x,y)] = max(lights[(x,y)] - 1, 0)

        elif line.startswith('turn on'):
            for x in range(x1, x2 + 1):
                for y in range(y1, y2 + 1):
                    # Turn on: Add one to existing brightness
                    lights[(x,y)] = lights[(x,y)] + 1

# Count up total brightness
print((sum(lights.values()))) #part 2

In [ ]:
#problem 7
import re
get_cmd = re.compile("[A-Z]+")
get_args = re.compile("[a-z0-9]+")

# Store functions by their name
funcs = {
    'AND': lambda a,b: a & b,
    'OR': lambda a,b: a | b,
    'LSHIFT': lambda a,b: a << b,
    'RSHIFT': lambda a,b: a >> b,
    'NOT': lambda a: ~a,
}

# Look up symbol, and its definition recursively, saving the results in 
# the wires dict.
def resolve(symbol):
    if isinstance(symbol, int):
        return symbol
    value = wires[symbol]
    if not isinstance(value, tuple):
        return value
    # Value must be a tuple (command, args)
    command, args = value
    if not command:
        # If no command, it must be a simple assignment
        result = resolve(args[0])
        # store result
        wires[symbol] = result
        return result
    else:
        resolved_args = [resolve(x) for x in args]
        result = funcs[command](*resolved_args)
        # store result
        wires[symbol] = result
        return result

wires = {}
with open('problem7.txt', 'r') as f:
    for line in f:
        # Parse the command
        command = get_cmd.search(line)
        # Get all the arguments via regex
        args = get_args.findall(line)
        # Convert numeric arguments to integers
        args = [int(x) if x.isdigit() else x for x in args]
        # Get result of search if we found anything
        if command:
            command = command.group()
        # Get the storage location of the command
        to_wire = args.pop()
        # Store it
        wires[to_wire] = (command, tuple(args))

#print(resolve('a')) #part 1


initial_wires = wires.copy()
result_a = resolve('a')
# Reset to initial state, but with 'b' changed.
wires = initial_wires
wires['b'] = result_a
print(resolve('a'))

        

In [ ]:
import sys

# Main
print(sum([len(line.strip()) - len(eval(line)) for line in open('problem8.txt')]))
print(sum([line.strip().count('\\') + line.strip().count('"') + 2 for line in open('problem8.txt')]))

In [ ]:
#problem 9
from itertools import permutations

distances = {}
places = set()

with open('problem9.txt', 'r') as f:
    for line in f:
        parts = line.split(' ')
        frm, _, to, _, amount = parts
        amount = int(amount.strip())
        distances[frm, to] = distances[to, frm] = amount
        places.add(frm)
        places.add(to)

possibilities = permutations(places)
totals = []

for route in possibilities:
    total = 0
    for i, frm in enumerate(route):
        if i == len(route) -1:
            break
        to = route[i+1]
        total += distances[(frm, to)]
    totals.append(total)
print(min(totals))
print(max(totals))

In [ ]:
#problem 10
import re
# This is a little complicated, we find a digit and capture all of its
# repetitions, in the second capture group we have the single digit itself
rep_digits = re.compile(r'((\d)\2*)')
looknsay = '3113322113'

for i in range(0,5):
    new_looknsay = ''
    for match, character in rep_digits.findall(looknsay):
        new_looknsay += str(len(match))
        new_looknsay += character
    looknsay = new_looknsay
    print(looknsay)
print(len(looknsay))

In [ ]:
#problem 11
import re


# Magic ASCII number
A = 97

# Can't have these
bad = re.compile(r'i|o|l')

# Must have two pairs
pairs = re.compile(r'([a-z])\1.*([a-z])\2')

# Check if we have a sequence of type 'abc'
def has_sequence(password):
    # Switch chars to numbers
    nums = list(map(ord, password))
    # This checks if the current three are a run
    has_run = lambda a,b,c: a == b - 1 and b == c - 1
    # This maps our has_run on offset sequences and let's us know if anythin
    # makes a run.
    return any(map(has_run, nums, nums[1:], nums[2:]))

def get_next_char(c):
    # Reduce number to 0-26 range, then add 1
    base_num = ord(c) - A + 1
    # Convert back to char, keeping within 0-26, then add back up to [a-z] range
    return chr((base_num % 26) + A)

def increment_string(s):
    if not s:
        return ''
    next_char = get_next_char(s[-1:])
    # If we overflow, recursively overflow on earlier substrings
    if next_char == 'a':
        return increment_string(s[:-1]) + 'a'
    next_string = s[:-1] + next_char
    return next_string

# Checks against all required predicates
def is_valid(password):
    return (has_sequence(current_password)
            and pairs.search(current_password)
            and not bad.search(current_password))

current_password = 'hxbxwxba'

while not is_valid(current_password):
    current_password = increment_string(current_password)
print(current_password) #part 1

current_password = increment_string(current_password)
while not is_valid(current_password):
    current_password = increment_string(current_password)
print(current_password) #part 2

In [ ]:
#problem 12
import re

p12 = re.compile(r'(-?\d+)') #find numbers
#part 1
print(sum([sum(int(x) for x in p12.findall(line)) for line in open('problem12.txt')]))

#part 2
import json

with open('problem12.txt') as file:
    filter_red = lambda x: {} if 'red'  in x.values() else x
    inp = json.load(file, object_hook=filter_red)

def sum_numbers_in_json(json):
    if isinstance(json, int):
        return json

    elif isinstance(json, list):
        return sum(sum_numbers_in_json(x) for x in json)

    elif isinstance(json, dict):
        return sum(sum_numbers_in_json(x) for x in json.values())
    return 0

print(sum_numbers_in_json(inp))



In [ ]:
#problem 13

import re
from collections import defaultdict
from itertools import permutations

person = re.compile(r'[A-Z][a-z]')
value = re.compile(r'-?\d+')
lose = re.compile(r'lose')
people = set()

mapping = defaultdict(int)
sorted_tuple = lambda x, y: tuple(sorted((x,y)))
with open('problem13.txt', 'r') as ins:
    for line in ins:
        sitter, neighbour = person.findall(line)
        diff = int(value.search(line).group())
        if lose.search(line):
            diff = -diff
        people.add(sitter)
        # Just get the total difference for the pair of people.
        mapping[sorted_tuple(sitter, neighbour)] += diff

totals = []

for p in permutations(people):
    total = 0
    for i, person in enumerate(p):
        # Just need each person and their previous person
        total += mapping[sorted_tuple(p[i-1], p[i]) ]
    totals.append(total)

print(max(totals))

In [12]:
#problem 14
#part 1
import re
p14 = re.compile("([0-9]+)")
max_dist = []
with open('problem14.txt', 'r') as ins:
    for line in ins:
        # Get the speed and times from the instruction via regex
        speed, duration, rest = [int(x) for x in p14.findall(line)]
        time = 2503
        traveled = 0
        while time > 0:
            if time >= duration:
                traveled += (speed * duration)
                time = time - (duration + rest)
            else:
                traveled += (speed * time)
                time = time - time
        max_dist.append(traveled)
print(max(max_dist))

2696


In [19]:
import re
import copy
p14 = re.compile("([0-9]+)")
dist_after_s = [0,0,0,0,0,0,0,0,0]
data = []
l = []
time = 2503
with open('problem14.txt', 'r') as ins:
    for line in ins:
        hl = []
        speed, duration, rest = [int(x) for x in p14.findall(line)]
        hl.append(speed), hl.append(duration), hl.append(rest)
        data.append(hl[:])
datac = copy.deepcopy(data)
while time > 0:
    for i in range(len(data)):
        if data[i][1] > 0:
            dist_after_s[i] += data[i][0]
            data[i][1]-=1
        elif data[i][1] == 0 and data[i][2] > 0:
            data[i][2]-=1
            if data[i][1] == 0 and data[i][2] == 0:
                data[i][1] = datac[i][1]
                data[i][2] = datac[i][2]
    time -= 1
    l.append(dist_after_s.index(max(dist_after_s))) # in each step append the index to a list with the max value for this step
print((max(set(l), key=l.count)),l.count(max(set(l), key=l.count))) # find the most common index nad # of it

0 1084
